# Detección de tweets meteorológicos con Bag of Words

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

#### Cargo la base de datos clasificada y genero las variables de importancia

In [48]:
TW = pd.read_csv('base_corr.csv')

In [49]:
TW.keys()[-3:]

Index(['Twit', 'Tag', 'Num'], dtype='object')

In [50]:
MSJ = TW['Twit']
TAG = TW['Tag']
NUM = TW['Num']

In [51]:
MSJ

0                                         Esta lloviendo
1      quiero ir al centro a la mañana pero seguro ll...
2                   @paloma_CAI7 Si, llueve y mucho !! 😒
3                       @rozalazar9 Si llueve les hago 🤗
4      Hasta cuándo le voy a seguir teniendo miedo a ...
5      13h ClimaRafaela: Temp. 20.9 °C, Hum. 48 %, Ba...
6      @Eze2450 Sea la estación que sea , no hay plat...
7      Llueve y no tengo Internet...me espera tremend...
8                            Que linda noche de lluvia 💖
9      Miércoles de mucho calor, previo a las lluvias...
10     Me gusta cuando llueve porque parece que se es...
11     Noviembre sin ti es sentir que la lluvia me di...
12     17h Agvnet CDU: Temp. 32.4 °C, Hum. 33 %, Bar....
13      Con esta lluvia me dieron ganas de cucharear ! 😏
14                               Y se largo la lluvia ☔💕
15                                  Que hermosa tormenta
16                         @joaquinv17 dan lluvia jajaja
17     Aseguran que hay stock s

#### Elimino las cosas innecesarias de msj

In [126]:
def normalizacion(mensaje):
    
    mensaje = list(mensaje)
    lenght = len(mensaje)
    
    lista_ascii = [['\\xe1', '\\xe9', '\\xed', '\\xf3', '\\xfa', '\\xf1', '\\xd1', '\\xc1','\\xc9', '\\xcd', '\\xd3', 
                 '\\xda', '\\xa1', '\\xbf', '\\xdc', '\\xfc', '\\xb4', '\\xba', '\\xaa', '\\xb7','\\xc7','\\xe7',
                 '\\xa8', '\\xb0C', '\\n', '\\xb0c', '\\xbb', 'xb0'],['a', 'e', 'i', 'o', 'u', 'ñ', 'Ñ', 'A', 'E', 'I', 'O', 'U', '', '', 'Ü', 'ü', '', 
                           ' ', '', '','Ç','ç','', ' ', ' ', ' ', ' ', ' ']]
    lista_simb = ['!','\'', '\"', "|", '$', "%", "&", "(", ")", "=", "?", "+",'/', ";", "_", "-", "1","2","3","4","5","6","7","8","9","0", '*',',', '.', ':', '#']
    
    ign_palabras = ['en','hpa','km','mm',"su","vos",'que',"re","xq","le","te","tu","soy","sos","mi","da","o","x","les","me","d","q", "como", "lo", "los", "mi", "son", "a", "el", 
                    "un","la", "una","en","por","para", 'las',"ante", "al", 'me',"rt", "del", "y", "se", "de", "que", "sus", "ha", "es", "con", "esta"]
    
    for i in range(lenght):
        
        ## Convierto mayúsculas en minúsculas
        mensaje[i] = mensaje[i].lower()
        
        ## Saco las menciones y otras cosas
        txt = mensaje[i].split()
        
        for j in range(len(txt)):
            if ('@' in txt[j]) and ('RELAMPAGO2018' not in txt[j]) and ('RELAMPAGO_edu' not in txt[j]) or ('jaj' in txt[j]) or ('https' in txt[j]): txt[j]=''
        
        mensaje[i] = " ".join(txt)
        
        ## Reemplazo símbolos
        for h in range(len(lista_simb)):
            mensaje[i] = mensaje[i].replace(lista_simb[h], ' ')
            
        ## Convierto el msj a ASCII, reemplazo los errores de decodificación y agrego un espacio antes de cada decodificación
        mensaje[i] = mensaje[i].encode('unicode-escape').decode('ASCII')+" "     
        
        for j in range(len(lista_ascii[0])):
            mensaje[i] = mensaje[i].replace(lista_ascii[0][j], lista_ascii[1][j])
        
        mensaje[i] = mensaje[i].replace('\\', ' \\')
        
        for j in range(len(ign_palabras)):
            mensaje[i] = mensaje[i].replace(" "+ign_palabras[j]+" ", ' ')
        
    return(mensaje)

In [127]:
msj = normalizacion(MSJ)

In [128]:
msj

['esta lloviendo ',
 'quiero ir centro mañana pero seguro llueve  ',
 ' si  llueve mucho     \\U0001f612 ',
 ' si llueve hago  \\U0001f917 ',
 'hasta cuando voy seguir teniendo miedo tormentas   \\U0001f644 \\U0001f623 ',
 '  h climarafaela  temp          hum        bar         mbar  lluvia      viento      h sso  weathercloud ',
 ' sea estacion sea   no hay plata  ahora llueve   cambia   tenes estudiar igual ',
 'llueve no tengo internet   espera tremenda tarde ',
 'que linda noche lluvia  \\U0001f496 ',
 'miercoles mucho calor  previo lluvias  ',
 'me gusta cuando llueve porque parece estan fritando empanadas  puesgorda  \\U0001f602 ',
 'noviembre sin ti sentir lluvia dice llorando todo acabo  \\U0001f3b5 ',
 '  h agvnet cdu  temp          hum        bar         lluvia    viento      h  weathercloud ',
 'con lluvia dieron ganas cucharear    \\U0001f60f ',
 'y largo lluvia  \\u2614 \\U0001f495 ',
 'que hermosa tormenta ',
 ' dan lluvia  ',
 'aseguran hay stock suficiente bengalas comb

#### División en datasets de  _train_ y _test_

In [129]:
size = 0.8

## Toma los tweets, los asocia con su tag y luego los divide en cada una de las palabras que los componen
## zip() itera tuplas
train = [(i.split(),j) for i,j in zip(list(msj)[1:int(len(msj)*0.8)],list(TAG)[1:int(len(msj)*0.8)])]

test = [(i.split(),j) for i,j in zip(list(msj)[int(len(msj)*0.8)+1:len(msj)],list(TAG)[int(len(msj)*0.8)+1:len(msj)])]

## No entiendo bien que es esto
label_to_ix = { 1: 0, 0: 1 }

In [142]:
word_to_ix = {}

for sent, _ in train+test:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

In [143]:
word_to_ix

{'quiero': 0,
 'ir': 1,
 'centro': 2,
 'mañana': 3,
 'pero': 4,
 'seguro': 5,
 'llueve': 6,
 'si': 7,
 'mucho': 8,
 '\\U0001f612': 9,
 'hago': 10,
 '\\U0001f917': 11,
 'hasta': 12,
 'cuando': 13,
 'voy': 14,
 'seguir': 15,
 'teniendo': 16,
 'miedo': 17,
 'tormentas': 18,
 '\\U0001f644': 19,
 '\\U0001f623': 20,
 'h': 21,
 'climarafaela': 22,
 'temp': 23,
 'hum': 24,
 'bar': 25,
 'mbar': 26,
 'lluvia': 27,
 'viento': 28,
 'sso': 29,
 'weathercloud': 30,
 'sea': 31,
 'estacion': 32,
 'no': 33,
 'hay': 34,
 'plata': 35,
 'ahora': 36,
 'cambia': 37,
 'tenes': 38,
 'estudiar': 39,
 'igual': 40,
 'tengo': 41,
 'internet': 42,
 'espera': 43,
 'tremenda': 44,
 'tarde': 45,
 'que': 46,
 'linda': 47,
 'noche': 48,
 '\\U0001f496': 49,
 'miercoles': 50,
 'calor': 51,
 'previo': 52,
 'lluvias': 53,
 'me': 54,
 'gusta': 55,
 'porque': 56,
 'parece': 57,
 'estan': 58,
 'fritando': 59,
 'empanadas': 60,
 'puesgorda': 61,
 '\\U0001f602': 62,
 'noviembre': 63,
 'sin': 64,
 'ti': 65,
 'sentir': 66,
 'dice

In [144]:
class BoWClassifier(nn.Module): # inheriting from nn.Module!
    
    def __init__(self, num_labels, vocab_size):
        
        super(BoWClassifier, self).__init__()

        self.linear = nn.Linear(vocab_size, num_labels)
        
        
    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return F.log_softmax(self.linear(bow_vec), dim=1)
    
    def make_bow_vector(sentence, word_to_ix):
        vec = torch.zeros(len(word_to_ix))
        for word in sentence:
            vec[word_to_ix[word]] += 1
        return vec.view(1, -1)

    def make_target(label, label_to_ix):
        
        return torch.LongTensor([label_to_ix[label]])
    
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [145]:
#print(BoWClassifier.make_bow_vector('me me gusta comer en la cafeteria'.split(),word_to_ix))


#for param in model.parameters():
    #print (param)
    
# To run the model, pass in a BoW vector, but wrapped in an autograd.Variable
sample = train[0]
#print (sample)
bow_vector = BoWClassifier.make_bow_vector(sample[0], word_to_ix)
#print(bow_vector)
log_probs = model(bow_vector)
#print (log_probs.exp())

In [146]:
# Run on test data before we train, just to see a before-and-after
for instance, label in test:
    bow_vec = BoWClassifier.make_bow_vector(instance, word_to_ix)
    log_probs = model(bow_vec)
    print (log_probs.exp(), label)
print (next(model.parameters())[:,word_to_ix["lluvia"]]) # Print the matrix column corresponding to "creo"

tensor([[0.5148, 0.4852]], grad_fn=<ExpBackward>) 0
tensor([[0.5120, 0.4880]], grad_fn=<ExpBackward>) 0
tensor([[0.5397, 0.4603]], grad_fn=<ExpBackward>) 0
tensor([[0.5017, 0.4983]], grad_fn=<ExpBackward>) 0
tensor([[0.5047, 0.4953]], grad_fn=<ExpBackward>) 0
tensor([[0.5143, 0.4857]], grad_fn=<ExpBackward>) 0
tensor([[0.5170, 0.4830]], grad_fn=<ExpBackward>) 0
tensor([[0.5163, 0.4837]], grad_fn=<ExpBackward>) 1
tensor([[0.5087, 0.4913]], grad_fn=<ExpBackward>) 1
tensor([[0.4974, 0.5026]], grad_fn=<ExpBackward>) 0
tensor([[0.5050, 0.4950]], grad_fn=<ExpBackward>) 0
tensor([[0.5427, 0.4573]], grad_fn=<ExpBackward>) 1
tensor([[0.4815, 0.5185]], grad_fn=<ExpBackward>) 0
tensor([[0.4852, 0.5148]], grad_fn=<ExpBackward>) 0
tensor([[0.4918, 0.5082]], grad_fn=<ExpBackward>) 0
tensor([[0.5317, 0.4683]], grad_fn=<ExpBackward>) 1
tensor([[0.5234, 0.4766]], grad_fn=<ExpBackward>) 1
tensor([[0.5203, 0.4797]], grad_fn=<ExpBackward>) 1
tensor([[0.4679, 0.5321]], grad_fn=<ExpBackward>) 0
tensor([[0.4

In [149]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Usually you want to pass over the training data several times.
# 100 is much bigger than on a real data set, but real datasets have more than
# two instances.  Usually, somewhere between 5 and 30 epochs is reasonable.
for epoch in range(500):
    if epoch % 10 == 0: print('Epoch: %i' %epoch)
    for instance, label in train:
        # Step 1. Remember that Pytorch accumulates gradients.  We need to clear them out
        # before each instance
        model.zero_grad()
    
        # Step 2. Make our BOW vector and also we must wrap the target in a Variable
        # as an integer.  For example, if the target is SPANISH, then we wrap the integer
        # 0.  The loss function then knows that the 0th element of the log probabilities is
        # the log probability corresponding to SPANISH
        bow_vec = BoWClassifier.make_bow_vector(instance, word_to_ix)
        target = BoWClassifier.make_target(label, label_to_ix)
    
        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)
    
        # Step 4. Compute the loss, gradients, and update the parameters by calling
        # optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

Epoch: 0
Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Epoch: 100
Epoch: 110
Epoch: 120
Epoch: 130
Epoch: 140
Epoch: 150
Epoch: 160
Epoch: 170
Epoch: 180
Epoch: 190
Epoch: 200
Epoch: 210
Epoch: 220
Epoch: 230
Epoch: 240
Epoch: 250
Epoch: 260
Epoch: 270
Epoch: 280
Epoch: 290
Epoch: 300
Epoch: 310
Epoch: 320
Epoch: 330
Epoch: 340
Epoch: 350
Epoch: 360
Epoch: 370
Epoch: 380
Epoch: 390
Epoch: 400
Epoch: 410
Epoch: 420
Epoch: 430
Epoch: 440
Epoch: 450
Epoch: 460
Epoch: 470
Epoch: 480
Epoch: 490


In [150]:
c = [0,0,0]
for instance, label in test:
    bow_vec = autograd.Variable(BoWClassifier.make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    log_probs_exp = log_probs.exp()
    if bool (log_probs_exp[0][0]>0.5) and (label == 1):
        c[0]+=1
        print ('\033[1;93m' , log_probs_exp , label)
    elif bool (log_probs_exp[0][1]>0.5) and (label == 0):
        c[1]+=1
        print ('\033[1;32m' , log_probs_exp , label)
    else:
        c[2]+=1
        print ('\033[1;31m' , log_probs_exp , label)
 
print('De aciertos Meteorológicos: ', round(c[0]),'%, De No Meteorologicos: ', round(c[1]), '%, de Fallas: ', round(c[2]),'%') 
print(c)
#print ('\033[0m', next(model.parameters())[:,word_to_ix["lluvia"]]) # Index corresponding to Spanish goes up, English goes down!

plt.bar(range(3),c)

 tensor([[0.1757, 0.8243]], grad_fn=<ExpBackward>) 0
 tensor([[0.5283, 0.4717]], grad_fn=<ExpBackward>) 0
 tensor([[0.4615, 0.5385]], grad_fn=<ExpBackward>) 0
 tensor([[0.1880, 0.8120]], grad_fn=<ExpBackward>) 0
 tensor([[0.1957, 0.8043]], grad_fn=<ExpBackward>) 0
 tensor([[0.1067, 0.8933]], grad_fn=<ExpBackward>) 0
 tensor([[0.5981, 0.4019]], grad_fn=<ExpBackward>) 0
 tensor([[0.0442, 0.9558]], grad_fn=<ExpBackward>) 1
 tensor([[0.8701, 0.1299]], grad_fn=<ExpBackward>) 1
 tensor([[0.4870, 0.5130]], grad_fn=<ExpBackward>) 0
 tensor([[0.0065, 0.9935]], grad_fn=<ExpBackward>) 0
 tensor([[0.6490, 0.3510]], grad_fn=<ExpBackward>) 1
 tensor([[0.3098, 0.6902]], grad_fn=<ExpBackward>) 0
 tensor([[0.6842, 0.3158]], grad_fn=<ExpBackward>) 0
 tensor([[0.0758, 0.9242]], grad_fn=<ExpBackward>) 0
 tensor([[0.9663, 0.0337]], grad_fn=<ExpBackward>) 1
 tensor([[0.5654, 0.4346]], grad_fn=<ExpBackward>) 1
 tensor([[0.3890, 0.6110]], grad_fn=<ExpBackward>) 1
 tensor([[0.3015, 0.6985]], grad_fn=<ExpBackwa

<BarContainer object of 3 artists>

In [151]:
test

[(['si',
   'llueve',
   'ahora',
   'menos',
   'espero',
   'mañana',
   'este',
   'nublado',
   'no',
   'aguanto',
   'calor',
   '\\U0001f612'],
  0),
 (['frio',
   '\\U0001f60a',
   'lluvia',
   '\\U0001f496',
   'azul',
   '\\U0001f631',
   'celular',
   '\\U0001f917',
   'solteras',
   '\\U0001f602',
   'mtvemabiggestfansbts'],
  0),
 (['aerolineas',
   'lluvia',
   'segun',
   'san',
   'idiota',
   'confirmado',
   'jacques',
   'cousteau',
   'era',
   'campora'],
  0),
 (['queria',
   'ir',
   'pileta',
   'llueve',
   'empece',
   'ver',
   'serie',
   'corta',
   'luz'],
  0),
 (['ultimamente', 'lluvia', 'ni', 'tiempo', 'nada', 'larga', 'todo', 'nomas'],
  0),
 (['quiero', 'alguien', 'pasar', 'dias', 'lluvia'], 0),
 (['chica', 'tormenta'], 0),
 (['lluvia', 'vida', 'amo', 'todo', 'corazon', 'gracias', 'mundo'], 1),
 (['que', 'linda', 'lluvia', '\\U0001f60d'], 1),
 (['encima', 'llueve', 'balde'], 0),
 (['lo', 'unico', 'voy', 'ver', 'mañana', 'tormenta', 'viniendo'], 0),
 (